In [1]:
from mcmc import *
from constants import *

We first run the coupled chains 1, 000 times independently in parallel. All observed meeting times were
less or equal to 4, so we consider k equal to 1, 3 and 5. Then, we consider m equal to k, 5k and 10k

In [2]:
K = len(z)

theta_0 = np.ones(shape = [K]) * np.mean(z)
h = lambda x: x[2][1]

In [3]:
theta_0

array([0.26566667, 0.26566667, 0.26566667, 0.26566667, 0.26566667,
       0.26566667, 0.26566667, 0.26566667, 0.26566667, 0.26566667,
       0.26566667, 0.26566667, 0.26566667, 0.26566667, 0.26566667,
       0.26566667, 0.26566667, 0.26566667])

# k = 5, m = 10 * k = 50

In [4]:
k = 5; m = 10 * k

In [5]:
res = unbiased_mcmc_coupled_gibbs_sampler(burnin = k, m = m, theta_0 = theta_0, h = h)

....Computing sums for GS
sum_of_squares:  1.2704179999999994
sum_element:  4.781999999999998
done;

sum_gamma, mean_mu:  2.220446049250313e-16 0.26566666666666655

....Simulating A
scale: 2.0
done;
....Simulating mu
mean_mu: 0.26566666666666655
done;
....Simulating theta
mean: 0.392408318147274
std: 0.004256453363462567
mean: 0.37279332568892115
std: 0.004256453363462567
mean: 0.3531783332305683
std: 0.004256453363462567
mean: 0.33258259114929783
std: 0.004256453363462567
mean: 0.31198684906802737
std: 0.004256453363462567
mean: 0.31198684906802737
std: 0.004256453363462567
mean: 0.2904103573638392
std: 0.004256453363462567
mean: 0.26883386565965117
std: 0.004256453363462567
mean: 0.24725737395546302
std: 0.004256453363462567
mean: 0.24725737395546302
std: 0.004256453363462567
mean: 0.22470013262835725
std: 0.004256453363462567
mean: 0.22470013262835725
std: 0.004256453363462567
mean: 0.22470013262835725
std: 0.004256453363462567
mean: 0.22470013262835725
std: 0.004256453363462567
mea

In [6]:
res

0.37247189529068014

In [ ]:
0.3930328741536556